# WSDM - KKBox's Music Recommendation Challenge  
#### Autores: Cristian Osorio, Jefry Cardona

## Descripción
No hace muchos años, era inconcebible que la misma persona escuchara a los Beatles, Vivaldi y Lady Gaga en su viaje matutino. Pero, los días de gloria de los DJs de radio han pasado, y los locutores y programadores musicales en la radio han sido reemplazados por algoritmos de personalización y servicios de streaming ilimitados.

Mientras que el público escucha todo tipo de música, los algoritmos siguen luchando en áreas clave. Sin suficientes datos históricos, ¿cómo sabría un algoritmo si a los oyentes les gustará una nueva canción o un nuevo artista? Y, ¿cómo sabría qué canciones recomendar a los nuevos usuarios?

El conjunto de datos es de KKBOX, el servicio de transmisión de música streaming líder en Asia, que contiene la biblioteca de música pop asiática más completa del mundo con más de 30 millones de pistas.

El trabajo para este proyecto es predecir la probabilidad de que un usuario escuche una canción repetedimante después de que se haya activado el primer evento de escucha observable dentro de un lapso de tiempo. Si hay eventos de escucha recurrentes que se activan dentro de un mes después del primer evento de escucha observable del usuario, su objetivo está marcado con un 1, y con un 0 en el conjunto de entrenamiento. La misma regla se aplica al conjunto de pruebas.

KKBOX proporciona un conjunto de datos de entrenamiento que consiste en información del primer evento de escucha observable para cada par de canciones de usuario único dentro de una duración de tiempo específica. También se proporcionan los metadatos de cada usuario y par de canciones único.

## Diccionario de datos

En principio el dataset se encuentra dividido en 6 archivos de tipo csv los cuales son: train, test, sample_submission, songs, members y song_extra_info. Cada uno contiene una serie de columnas de datos que se explicarán a continuación:

### train.csv

* *msno*: id del usuario
* *song_id*: id de la canción.
* *source_system_tab*: el nombre de la pestaña en la que se ha lannzado el evento. Las pestañas de sistema se utilizan para categorizar las funciones de las aplicaciones móviles de KKBOX. Por ejemplo, la pestaña ```my library``` contiene funciones para manipular el almacenamiento local, y la pestañas ```search``` contiene funciones relacionadas con la búsqueda.
* *source_screen_name*: nombre de la pantalla que el usuario está viendo.
* *source_type*: punto de entrada en el que el usuario reproduce por primera vez música en aplicaciones móviles. Un punto de entrada podría ser un álbum, una lista de reproducción en línea, una canción... etc.
* *target*: es la variable objetivo, ```target=1``` significa que hay eventos de escucha recurrentes que se activan en el plazo de un mes después del primer evento de escucha observable del usuario, ```target=1``` en caso contrario.

### test.csv

* *id*: será usado para submission.
* *msno*: id del usuario.
* *song_id*: id de la canción.
* *source_system_tab*: el nombre de la pestaña en la que se ha lannzado el evento. Las pestañas de sistema se utilizan para categorizar las funciones de las aplicaciones móviles de KKBOX. Por ejemplo, la pestaña ```my library``` contiene funciones para manipular el almacenamiento local, y la pestañas ```search``` contiene funciones relacionadas con la búsqueda.
* *source_screen_name*: nombre de la pantalla que el usuario está viendo.
* *source_type*: punto de entrada en el que el usuario reproduce por primera vez música en aplicaciones móviles. Un punto de entrada podría ser un álbum, una lista de reproducción en línea, una canción... etc.

### sample_submission.csv

Formato del archivo en la que se espera se envíen los resultados a la competencia.

* *id*: mismo id del test.csv.
* *target*: es la variable objetivo. ```target=1``` significa que hay eventos de escucha recurrentes que se activan dentro de un mes después del primer evento de escucha observable del usuario, ```target=0``` en caso contrario.

### songs.csv

* *song_id*: id de la canción.
* *song_length*: duración de la canción en milisegundos (ms).
* *genre_ids*: categoría de genero. Algunas canciones tienen multiples generos y estan separados mediante "|".
* *artist_name*: nombre del artista.
* *composer*: compositor.
* *lyricist*: letrista.
* *language*: lenguaje.

### members.csv

* *msno*: id del usuario.
* *city*: ciudad del miembro.
* *bd*: años de edad del miembro.
* *gender*: género.
* *registered_via*: método de registro.
* *registration_init_time*: fecha de registro en formato ```%Y%m%d```.
* *expiration_date*: fecha de expiración en formato ```%Y%m%d```.

### song_extra_info.csv

* *song_id*: id de la canción.
* *song_name*: nnombre de la canción.
* *isrc*: (International Standart Recording Code), teóricamente puede ser usado como un identificador de una canción. Sin embargo, cabe señalar que los ISRC generados por los proveedores no han sido verificados oficialmente; por lo tanto, la información del ISRC, como el código de país y el año de referencia, puede ser engañosa o incorrecta. Múltiples canciones podrían compartir un ISRC ya que una sola grabación podría volver a publicarse varias veces.

## Análisis e identificación de problemas de los datos

In [1]:
import numpy as np #operaciones matriciales y con vectores
import pandas as pd #tratamiento de datos
import matplotlib.pyplot as plt #gráficos
from sklearn.model_selection import train_test_split #metodo de particionamiento de datasets para evaluación
from sklearn.model_selection import KFold, cross_val_score #protocolo de evaluación
from sklearn import neighbors, datasets, metrics
from sklearn import preprocessing 
import seaborn as sns
import csv

Cargaremos los datos en memoria y los mostraremos resumidos para encontrar problemas iniciales.

In [2]:
train = pd.read_csv(r"D:\2018-2\Data-Mining\dataset proyecto\train.csv", sep=',')
train.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7377418 entries, 0 to 7377417
Data columns (total 6 columns):
msno                  object
song_id               object
source_system_tab     object
source_screen_name    object
source_type           object
target                int64
dtypes: int64(1), object(5)
memory usage: 337.7+ MB


In [4]:
train.describe(include="all")

,msno,song_id,source_system_tab,source_screen_name,source_type,target
count,7377418,7377418,7352569,6962614,7355879,7.377418e+06
unique,30755,359966,8,20,12,NaN
top,MXIMDXO0j3UpaT7FvOSGW6Y5zfhlh+xYjTqGoUdMzEE=,reXuGcEWDDCnL0K3Th//3DFG4S1ACSpJMzA+CFipo1g=,my library,Local playlist more,local-library,NaN
freq,5819,13973,3684730,3228202,2261399,NaN
mean,NaN,NaN,NaN,NaN,NaN,5.035171e-01
std,NaN,NaN,NaN,NaN,NaN,4.999877e-01
min,NaN,NaN,NaN,NaN,NaN,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,0.000000e+00
50%,NaN,NaN,NaN,NaN,NaN,1.000000e+00
75%,NaN,NaN,NaN,NaN,NaN,1.000000e+00


In [5]:
train.dtypes

msno                  object
song_id               object
source_system_tab     object
source_screen_name    object
source_type           object
target                 int64
dtype: object

In [6]:
test = pd.read_csv(r"D:\2018-2\Data-Mining\dataset proyecto\test.csv", sep=',')
test.head()

,id,msno,song_id,source_system_tab,source_screen_name,source_type
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,NaN,song-based-playlist
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,Radio,radio
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,Radio,radio


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556790 entries, 0 to 2556789
Data columns (total 6 columns):
id                    int64
msno                  object
song_id               object
source_system_tab     object
source_screen_name    object
source_type           object
dtypes: int64(1), object(5)
memory usage: 117.0+ MB


In [8]:
test.describe(include="all")

,id,msno,song_id,source_system_tab,source_screen_name,source_type
count,2.556790e+06,2556790,2556790,2548348,2393907,2549493
unique,NaN,25131,224753,8,22,12
top,NaN,KGXNZ/H3VxvET/+rGxlrAe7Gpz2eKMXyuSg3xh8Ij1M=,ZcKgNis1AP1LA0sdtIddrtk7P04iiJzJrXvwXdT/X3Q=,my library,Local playlist more,online-playlist
freq,NaN,2489,8320,1019492,845115,774532
mean,1.278394e+06,NaN,NaN,NaN,NaN,NaN
std,7.380818e+05,NaN,NaN,NaN,NaN,NaN
min,0.000000e+00,NaN,NaN,NaN,NaN,NaN
25%,6.391972e+05,NaN,NaN,NaN,NaN,NaN
50%,1.278394e+06,NaN,NaN,NaN,NaN,NaN
75%,1.917592e+06,NaN,NaN,NaN,NaN,NaN


In [9]:
test.dtypes

id                     int64
msno                  object
song_id               object
source_system_tab     object
source_screen_name    object
source_type           object
dtype: object

In [10]:
sample_sub = pd.read_csv(r"D:\2018-2\Data-Mining\dataset proyecto\sample_submission.csv", sep=',')
sample_sub.head()

,id,target
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [11]:
sample_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556790 entries, 0 to 2556789
Data columns (total 2 columns):
id        int64
target    float64
dtypes: float64(1), int64(1)
memory usage: 39.0 MB


In [12]:
sample_sub.describe(include="all")

,id,target
count,2.556790e+06,2556790.0
mean,1.278394e+06,0.5
std,7.380818e+05,0.0
min,0.000000e+00,0.5
25%,6.391972e+05,0.5
50%,1.278394e+06,0.5
75%,1.917592e+06,0.5
max,2.556789e+06,0.5


In [13]:
sample_sub.dtypes

id          int64
target    float64
dtype: object

In [14]:
songs = pd.read_csv(r"D:\2018-2\Data-Mining\dataset proyecto\songs.csv", sep=',')
songs.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [15]:
songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2296320 entries, 0 to 2296319
Data columns (total 7 columns):
song_id        object
song_length    int64
genre_ids      object
artist_name    object
composer       object
lyricist       object
language       float64
dtypes: float64(1), int64(1), object(5)
memory usage: 122.6+ MB


In [16]:
songs.describe(include="all")

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
count,2296320,2.296320e+06,2202204,2296320,1224966,351052,2.296319e+06
unique,2296320,NaN,1045,222363,329823,110925,NaN
top,LuKHGW+OLXu71ieZsk7fX3i5NGG5NBH70t1c+C9TkS8=,NaN,465,Various Artists,Neuromancer,Traditional,NaN
freq,1,NaN,567911,145916,17888,1751,NaN
mean,NaN,2.469935e+05,NaN,NaN,NaN,NaN,3.237800e+01
std,NaN,1.609200e+05,NaN,NaN,NaN,NaN,2.433241e+01
min,NaN,1.850000e+02,NaN,NaN,NaN,NaN,-1.000000e+00
25%,NaN,1.836000e+05,NaN,NaN,NaN,NaN,-1.000000e+00
50%,NaN,2.266270e+05,NaN,NaN,NaN,NaN,5.200000e+01
75%,NaN,2.772690e+05,NaN,NaN,NaN,NaN,5.200000e+01


In [17]:
songs.dtypes

song_id         object
song_length      int64
genre_ids       object
artist_name     object
composer        object
lyricist        object
language       float64
dtype: object

In [81]:
members = pd.read_csv(r"D:\2018-2\Data-Mining\dataset proyecto\members.csv", sep=',')
members.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,20170126,20170613


In [82]:
members.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34403 entries, 0 to 34402
Data columns (total 7 columns):
msno                      34403 non-null object
city                      34403 non-null int64
bd                        34403 non-null int64
gender                    14501 non-null object
registered_via            34403 non-null int64
registration_init_time    34403 non-null int64
expiration_date           34403 non-null int64
dtypes: int64(5), object(2)
memory usage: 1.8+ MB


In [83]:
members.describe(include="all")

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
count,34403,34403.000000,34403.000000,14501,34403.000000,3.440300e+04,3.440300e+04
unique,34403,NaN,NaN,2,NaN,NaN,NaN
top,P7ogzonQfQ+ZBjjRcQXpxpixh3CgUES9os8nmHCFoQQ=,NaN,NaN,male,NaN,NaN,NaN
freq,1,NaN,NaN,7405,NaN,NaN,NaN
mean,NaN,5.371276,12.280935,NaN,5.953376,2.013994e+07,2.016901e+07
std,NaN,6.243929,18.170251,NaN,2.287534,2.954015e+04,7.320925e+03
min,NaN,1.000000,-43.000000,NaN,3.000000,2.004033e+07,1.970010e+07
25%,NaN,1.000000,0.000000,NaN,4.000000,2.012103e+07,2.017020e+07
50%,NaN,1.000000,0.000000,NaN,7.000000,2.015090e+07,2.017091e+07
75%,NaN,10.000000,25.000000,NaN,9.000000,2.016110e+07,2.017093e+07


In [84]:
members.dtypes

msno                      object
city                       int64
bd                         int64
gender                    object
registered_via             int64
registration_init_time     int64
expiration_date            int64
dtype: object

In [22]:
song_extra_info = pd.read_csv(r"D:\2018-2\Data-Mining\dataset proyecto\song_extra_info.csv", sep=',')
song_extra_info.head()

,song_id,name,isrc
0,LP7pLJoJFBvyuUwvu+oLzjT+bI+UeBPURCecJsX1jjs=,我們,TWUM71200043
1,ClazTFnk6r0Bnuie44bocdNMM3rdlrq0bCGAsGUWcHE=,Let Me Love You,QMZSY1600015
2,u2ja/bZE3zhCGxvbbOB3zOoUjx27u40cf5g09UXMoKQ=,原諒我,TWA530887303
3,92Fqsy0+p6+RHe2EoLKjHahORHR1Kq1TBJoClW9v+Ts=,Classic,USSM11301446
4,0QFmz/+rJy1Q56C1DuYqT9hKKqi5TUqx0sN0IwvoHrw=,愛投羅網,TWA471306001


In [23]:
song_extra_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2295971 entries, 0 to 2295970
Data columns (total 3 columns):
song_id    object
name       object
isrc       object
dtypes: object(3)
memory usage: 52.6+ MB


In [24]:
song_extra_info.describe(include="all")

,song_id,name,isrc
count,2295971,2295969,2159423
unique,2295971,1168979,1806825
top,p2s2ay9Z/k+lOlYARzi5TzT62lZvzFOAI153Rg4ANqQ=,Intro,GBPS81518952
freq,1,1734,207


In [25]:
song_extra_info.dtypes

song_id    object
name       object
isrc       object
dtype: object

En un primer análisis hemos encontrado los siguiente problemas:
* La tabla ```sample_submision``` sólo sirve como referencia para la entrega de los resultados de predicción a los creadores de la competencia, por lo tanto podemos prescindir de ella ya que no aportará información al modelo.
* En la tabla ```train``` podemos observar que la variable objetivo llamada ```target``` se encuentra con tipo de dato ```int64```, lo que nos generará que el modelo realice una regresión lineal y no una clasificación que es lo que se nos pide.
* En la tabla ```members``` la columna ```bd``` que representa los años del miembro cuando revisamos el valor minimo y el máximo vemos que los datos no son lógicos al ser el minino un número negativo y el máximo 1051 años, lo que nos indica la existencia de datos atípicos. Tambien analizando los primeros datos arrojados por la función head() visualizamos que los primeros registros de la columna gender son nulos por lo cual se deberá aplicarle algún tratamiento para manejarlos.

## Tratamiento de los datos

Convertiremos la variable objetivo ```target```de la tabla ```train``` que se encuentra como contínua a categórica.

In [26]:
train['target'] = train['target'].astype(object)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7377418 entries, 0 to 7377417
Data columns (total 6 columns):
msno                  object
song_id               object
source_system_tab     object
source_screen_name    object
source_type           object
target                object
dtypes: object(6)
memory usage: 337.7+ MB


Se eliminarán todas las filas de la tabla ```train``` que contengan valores nulos.

In [27]:
train = train.dropna()
train.isna()[:5]

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False


Se eliminarán todas las filas de la tabla ```test``` que contengan valores nulos.

In [28]:
test = test.dropna()
test.isna()[:5]

,id,msno,song_id,source_system_tab,source_screen_name,source_type
0,False,False,False,False,False,False
1,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
5,False,False,False,False,False,False


Se eliminarán todas las filas de la tabla ```songs``` que contengan valores nulos.

In [29]:
songs = songs.dropna()
songs.isna()[:5]

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False


Se eliminarán todas las filas de la tabla ```members``` que contengan valores nulos. 

In [85]:
members = members.dropna()
members.isna()[:5]

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
5,False,False,False,False,False,False,False
10,False,False,False,False,False,False,False
13,False,False,False,False,False,False,False
15,False,False,False,False,False,False,False
16,False,False,False,False,False,False,False


Para los datos de los años negativos en la tabla ```members``` se les aplicará la función valor absoluto paa covertirlos a positivos y como tenemos datos de años que pueden llegar a los 1051, entonces nos guiaremos por información de la Organización mundial de la Salud la cual dice que la media de la esperanza de vida mundial es de 72 años y como tenemos una desviación estándar de 18 años entonces tomaremos como nuestra edad más alta los 90 años.

In [86]:
members['bd'] = members['bd'].abs()
members = members[members.bd <= 90]
members.describe(include="all")

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
count,14475,14475.000000,14475.000000,14475,14475.000000,1.447500e+04,1.447500e+04
unique,14475,NaN,NaN,2,NaN,NaN,NaN
top,8qw68ah05sifYivYtlLp9GXihPxK26aC8sol6UO2EVc=,NaN,NaN,male,NaN,NaN,NaN
freq,1,NaN,NaN,7388,NaN,NaN,NaN
mean,NaN,10.369050,27.984180,NaN,6.522694,2.012270e+07,2.017016e+07
std,NaN,6.091777,10.278099,NaN,2.680274,3.342326e+04,8.192021e+03
min,NaN,1.000000,0.000000,NaN,3.000000,2.004033e+07,2.004102e+07
25%,NaN,5.000000,22.000000,NaN,3.000000,2.010121e+07,2.017071e+07
50%,NaN,12.000000,27.000000,NaN,7.000000,2.013030e+07,2.017092e+07
75%,NaN,14.000000,33.000000,NaN,9.000000,2.015080e+07,2.017101e+07


In [87]:
members.replace({'bd' : [0,1,2,3]}, 10, inplace=True)
members.describe(include="all")

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
count,14475,14475.000000,14475.000000,14475,14475.000000,1.447500e+04,1.447500e+04
unique,14475,NaN,NaN,2,NaN,NaN,NaN
top,8qw68ah05sifYivYtlLp9GXihPxK26aC8sol6UO2EVc=,NaN,NaN,male,NaN,NaN,NaN
freq,1,NaN,NaN,7388,NaN,NaN,NaN
mean,NaN,10.369050,28.296788,NaN,6.522694,2.012270e+07,2.017016e+07
std,NaN,6.091777,9.548459,NaN,2.680274,3.342326e+04,8.192021e+03
min,NaN,1.000000,5.000000,NaN,3.000000,2.004033e+07,2.004102e+07
25%,NaN,5.000000,22.000000,NaN,3.000000,2.010121e+07,2.017071e+07
50%,NaN,12.000000,27.000000,NaN,7.000000,2.013030e+07,2.017092e+07
75%,NaN,14.000000,33.000000,NaN,9.000000,2.015080e+07,2.017101e+07


In [88]:
members['registration_init_time'] = members['registration_init_time'].astype(str)
members['expiration_date'] = members['expiration_date'].astype(str)
members.registration_init_time = members.registration_init_time.str[:-4]
members.expiration_date = members.expiration_date.str[:-4]
members.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
5,zgPOEyUn5a/Fvuzb3m69ajzxjkbblVtObglW89FzLdo=,13,43,female,9,2012,2017
10,GoFYKhcq8Q5Gjz1M5L0azHZOhcY+Za/T7fRqIGgBuYA=,12,28,male,9,2006,2017
13,RoSfblbwJN/izEnFIVw8TgOpm8R/NEpUC84Oz/b32HQ=,15,33,female,3,2015,2017
15,vUzJAyFEudsnkWwfcLIKT5mJAV+uVRTW2uajCrghFwI=,13,20,male,3,2014,2017
16,Ev+ouoRTpAjrtxNVeThP6t9xnDG1puQaG28DvJ2db+A=,13,30,male,9,2007,2017


Se eliminará la columna ```isrc```  de la tabla ```song_extra_info``` ya que puede llegar a contener información erronea o duplicada por lo que nos podría llevar a resultados no esperados.

In [32]:
song_extra_info = song_extra_info.loc[:,['song_id', 'name']]
song_extra_info.head()

,song_id,name
0,LP7pLJoJFBvyuUwvu+oLzjT+bI+UeBPURCecJsX1jjs=,我們
1,ClazTFnk6r0Bnuie44bocdNMM3rdlrq0bCGAsGUWcHE=,Let Me Love You
2,u2ja/bZE3zhCGxvbbOB3zOoUjx27u40cf5g09UXMoKQ=,原諒我
3,92Fqsy0+p6+RHe2EoLKjHahORHR1Kq1TBJoClW9v+Ts=,Classic
4,0QFmz/+rJy1Q56C1DuYqT9hKKqi5TUqx0sN0IwvoHrw=,愛投羅網


Ahora eliminaremos las filas que contengan valores nulos en la misma anterior.

In [33]:
song_extra_info = song_extra_info.dropna()
song_extra_info.isna()[:5]

,song_id,name
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
